# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [ ]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

---
### 1.4. Instalar librerías necesarias

In [ ]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow-gpu==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [2]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

print(gym.__version__)

0.17.3


#### Configuración base

In [3]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [4]:
from tensorflow.python.client import device_lib
from tensorflow import config

print(config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [6]:
import keyboard  # Para detectar teclas del teclado
import pygame  # Para el manejo de ventanas gráficas y la interacción
import time

# Inicializar el entorno
env.reset()

# Ejecutar el entorno
done = False
step = 0
while not done:

    action = 0  # Acción por defecto (no hacer nada)
    
    # Movimientos y disparo
    if keyboard.is_pressed('left'):  # Tecla de izquierda
        action = 3  # Mover a la izquierda
    elif keyboard.is_pressed('right'):  # Tecla de derecha
        action = 2  # Mover a la derecha
    elif keyboard.is_pressed('space'):  # Tecla espacio para disparar
        action = 1  # Disparar

    # Ejecuta la acción en el entorno
    observation, reward, done, info = env.step(action)
    
    # Renderiza el entorno
    env.render()
    time.sleep(0.05)
    
# Cerrar el entorno después de que termine
env.close()

pygame 2.6.1 (SDL 2.28.4, Python 3.8.20)
Hello from the pygame community. https://www.pygame.org/contribute.html


1. Implementación de la red neuronal

In [7]:
model = Sequential()
model.add(Permute((2,3,1), input_shape = (4,84,84)))
model.add(Convolution2D(32, (8,8), strides = (4,4), activation = 'relu'))
model.add(Convolution2D(64,(4,4), strides = (2,2), activation = 'relu'))
model.add(Convolution2D(64,(3,3), strides = (1,1), activation = 'relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))

2. Implementación de la solución DQN

In [12]:

processor = AtariProcessor()
memory = SequentialMemory(limit=1000000, window_length = 4)
policy = EpsGreedyQPolicy()

DQN = DQNAgent(model=model,
               nb_actions = nb_actions,
               memory = memory,
               nb_steps_warmup = 25000,
               target_model_update = 10000,
               policy = policy,
               processor = processor)

DQN.compile(Adam(learning_rate = 1e-4),metrics=['mae'])  


In [ ]:
DQN.fit(env, nb_steps=500000, visualize=False, verbose=2)
DQN.save_weights('dqn_v0_weights.h5f', overwrite=True)

Training for 500000 steps ...


c:\Users\DavidV\anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    693/500000: episode: 1, duration: 3.100s, episode steps: 693, steps per second: 224, episode reward:  5.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 2.964 [0.000, 5.000],  loss: --, mae: --, mean_q: --
   1382/500000: episode: 2, duration: 2.942s, episode steps: 689, steps per second: 234, episode reward:  4.000, mean reward:  0.006 [ 0.000,  1.000], mean action: 2.943 [0.000, 5.000],  loss: --, mae: --, mean_q: --
   2062/500000: episode: 3, duration: 2.915s, episode steps: 680, steps per second: 233, episode reward:  5.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 2.921 [0.000, 5.000],  loss: --, mae: --, mean_q: --
   2769/500000: episode: 4, duration: 3.018s, episode steps: 707, steps per second: 234, episode reward:  6.000, mean reward:  0.008 [ 0.000,  1.000], mean action: 2.926 [0.000, 5.000],  loss: --, mae: --, mean_q: --
   3441/500000: episode: 5, duration: 2.870s, episode steps: 672, steps per second: 234, episode reward:  5.000, mean reward:  0.007

c:\Users\DavidV\anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  25150/500000: episode: 35, duration: 15.498s, episode steps: 716, steps per second:  46, episode reward:  6.000, mean reward:  0.008 [ 0.000,  1.000], mean action: 2.797 [0.000, 5.000],  loss: 0.004939, mae: 0.040935, mean_q: 0.067121
  26169/500000: episode: 36, duration: 80.161s, episode steps: 1019, steps per second:  13, episode reward: 14.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.698 [0.000, 5.000],  loss: 0.003392, mae: 0.042725, mean_q: 0.064430
  26950/500000: episode: 37, duration: 61.781s, episode steps: 781, steps per second:  13, episode reward: 12.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.690 [0.000, 5.000],  loss: 0.003857, mae: 0.044833, mean_q: 0.064581
  27666/500000: episode: 38, duration: 57.243s, episode steps: 716, steps per second:  13, episode reward: 12.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 2.640 [0.000, 5.000],  loss: 0.004068, mae: 0.045858, mean_q: 0.068504
  28283/500000: episode: 39, duration: 49.546s, epi

In [ ]:
DQN.save_weights('models/dqn_v0_weights.h5f', overwrite=True)

In [ ]:
# Testing part to calculate the mean reward
weights_filename = 'models/dqn_v0_weights.h5f'.format(env_name)
DQN.load_weights(weights_filename)
DQN.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 9.000, steps: 486
Episode 2: reward: 11.000, steps: 540
Episode 3: reward: 15.000, steps: 712
Episode 4: reward: 14.000, steps: 691
Episode 5: reward: 18.000, steps: 766
Episode 6: reward: 20.000, steps: 875
Episode 7: reward: 26.000, steps: 1163
Episode 8: reward: 13.000, steps: 624
Episode 9: reward: 15.000, steps: 676
Episode 10: reward: 19.000, steps: 749


3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---